# Transormacion de datos

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import os



In [3]:
df = pd.read_csv('../csv/Injuries.csv')
df.head()

,Date,Team,Acquired,Relinquished,Notes
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely)


In [338]:
df['Date'] = pd.to_datetime(df['Date'])

In [339]:
#Posterior a 2013
df = df[df['Date'].dt.year >= 2013]

In [410]:
# Placed on IL significa que el jugador fue puesto en la lista de lesionados
df[df['Notes'] == 'placed on IL'].head(5)

,Date,Team,Acquired,Relinquished,Notes,Out Indefinitely,injury_type,body_part,SEASON


In [341]:
df = df[df['Notes'] != 'placed on IL']
df = df[df['Notes'] != 'activated from IL']
df = df[df['Notes'] != 'returned to lineup']

# Quita todos los que hayan vuelto de una lesion

In [342]:
df['Out Indefinitely'] = df['Notes'].str.contains('out indefinitely') | df['Notes'].str.contains('out for season')
# Crea una columna para aquellos que esten fuera indefinidamente o por toda la temporada

In [411]:
df[df['Notes'] =='placed on IL with sprained left ankle'].head(5)
# Esguinces de tobillo izquierdo

,Date,Team,Acquired,Relinquished,Notes,Out Indefinitely,injury_type,body_part,SEASON
7547,2013-10-30,Cavaliers,NaN,Sergey Karasev,placed on IL with sprained left ankle,False,ankle,Leg,13-14
7857,2013-11-18,Nets,NaN,Brook Lopez,placed on IL with sprained left ankle,False,ankle,Leg,13-14
7939,2013-11-23,Magic,NaN,Tobias Harris,placed on IL with sprained left ankle,False,ankle,Leg,13-14
8422,2013-12-23,Hawks,NaN,Cartier Martin,placed on IL with sprained left ankle,False,ankle,Leg,13-14
8550,2014-01-02,Magic,NaN,Solomon Jones,placed on IL with sprained left ankle,False,ankle,Leg,13-14


In [344]:
'''
En este fragmento de código, se define una lista llamada `injuries` que contiene una serie de lesiones 
relacionadas con el baloncesto. Luego, se define una función llamada `injury_detection` que recibe un parámetro `x`. 
Esta función busca coincidencias entre las lesiones de la lista y el parámetro `x`, y devuelve el resultado 
de la detección de lesiones.
La función recorre cada elemento de la lista `injuries` y verifica si ese elemento está presente en `x`. 
Si encuentra una coincidencia, agrega la lesión a una lista llamada `matching_injury`. 
Luego, se verifica la longitud de la lista `matching_injury`. Si solo hay una lesión coincidente, 
se devuelve esa lesión. Si hay más de una lesión coincidente, se devuelve una cadena que las une con la palabra "y". 
Si no hay lesiones coincidentes, se devuelve la cadena "other".
'''
injuries = ['knee', 'shoulder', 'hip', 'ankle', 'elbow', 'foot', 'heel', 'hand', 'hamstring', 'back', 'groin', 'neck', 'head', 'wrist', 'calf', 'leg', 'concussion', 'Achilles',
            'thigh', 'tibia', 'toe', 'adductor', 'chest', 'nose', 'forearm', 'gluteus', 'eye', 'patella', 'quadricap','rib','shin','pelvis','kneecap','orbital','thumb',
            'spinal cord', 'tailbone','cervical','cornea','cheekbone','collarbone','jaw','flexor','forehead','fibula','oral','facial','abdominal','abdomen','heart','throat', 'quadricep', 'oblique', 'finger', 'hernia'
            , 'abductor', 'tricep', 'bicep', 'lat', 'lung', 'face','biceps','triceps','pectoral','plantar','disc', 'adductor', 'bronchitis','chest','nose','forearm','gluteus','eye','heel','patella',
            'quadricap','rib,shin', 'thigh','tibia','toe','pelvis','kneecap','leg','orbital bone','thumb','spinal cord','tailbone','cervical','corne']

def injury_detection(x):
  matching_injury = []
  for injury in injuries:
    if injury in x:
      matching_injury.append(injury)
  if len(matching_injury) == 1:
    return matching_injury[0]
  elif len(matching_injury) > 1:
    return ' and '.join(matching_injury)
  elif len(matching_injury) == 0:
    return 'other'


In [345]:
#Se crea la columna para asignar el tipo de lesion utilizando la funcion injury_detection
df['injury_type'] = df['Notes'].apply(injury_detection)

In [346]:
# Cantidad de lesiones por tipo
df['injury_type'].value_counts()

injury_type
knee                                                     1871
other                                                    1737
ankle                                                    1651
back                                                      646
hamstring                                                 506
foot                                                      423
shoulder                                                  388
calf                                                      293
hip                                                       276
groin                                                     206
concussion                                                205
Achilles                                                  190
wrist                                                     175
hand                                                      170
quadricep                                                 159
toe and toe                                               

In [347]:
# Funcion para categorizar las lesiones
def categorize_injury(injury):
    if injury in ['hand', 'wrist', 'thumb', 'finger']:
        return 'Hand'
    elif injury in ['facial', 'jaw', 'cheekbone', 'orbital', 'nose', 'eye', 'oral', 'cornea', 'forehead','throat','face']:
        return 'Facial'
    elif injury in ['back', 'spinal cord', 'tailbone']:
        return 'Back'
    elif injury in ['leg', 'thigh', 'tibia', 'shin', 'quadricep', 'hamstring', 'knee','patella', 'kneecap', 'calf','Achilles','adductor','fibula', 'ankle']:
        return 'Leg'
    elif injury in ['plantar','foot', 'heel', 'toe']:
        return 'Foot'
    elif injury in ['forearm','elbow', 'arm','tricep','bicep','biceps','triceps','flexor']:
        return 'Arm'
    elif injury in ['shoulder', 'chest', 'collarbone', 'rib','abdominal', 'abdomen', 'heart', 'lung', 'pectoral','disc']:
        return 'Upper Body'
    elif injury in ['hip', 'groin', 'pelvis', 'gluteus','quadricap']:
        return 'Lower Body'
    elif injury in ['head','concussion', 'neck','cervical']:
        return 'Head'
    else:
        return 'Others'

# Aplicar la categorizacion para crear la columna body_part
df['body_part'] = df['injury_type'].apply(categorize_injury)


In [348]:
# Categoriza la temporada en la que se encuentra, es decir si esta entre dos fechas años, por ejemplo 2015-04-01 se catgoriza como 14-15
def categorize_season(date):
    if pd.Timestamp('2012-10-30') <= date <= pd.Timestamp('2013-10-28'):
        return '12-13'
    elif pd.Timestamp('2013-10-29') <= date <= pd.Timestamp('2014-10-27'):
        return '13-14'
    elif pd.Timestamp('2014-10-28') <= date <= pd.Timestamp('2015-10-26'):
        return '14-15'
    elif pd.Timestamp('2015-10-27') <= date <= pd.Timestamp('2016-10-24'):
        return '15-16'
    elif pd.Timestamp('2016-10-24') <= date <= pd.Timestamp('2017-10-16'):
        return '16-17'
    elif pd.Timestamp('2017-10-17') <= date <= pd.Timestamp('2018-10-15'):
        return '17-18'
    elif pd.Timestamp('2018-10-16') <= date <= pd.Timestamp('2019-10-21'):
        return '18-19'
    elif pd.Timestamp('2019-10-22') <= date <= pd.Timestamp('2020-12-21'):
        return '19-20'
    elif pd.Timestamp('2020-12-22') <= date <= pd.Timestamp('2021-10-18'):
        return '20-21'

In [349]:
df['SEASON'] = df['Date'].apply(categorize_season)

In [350]:
# Filtrar solo por 2013 en adelante

df = df[df['SEASON'].notna()]
df = df[~df['SEASON'].isin(['10-11', '11-12', '12-13'])]

In [351]:
df_injuries_only = df.copy()

In [412]:
df_injuries_only.head()

,Date,Team,Acquired,Relinquished,Notes,Out Indefinitely,injury_type,body_part,SEASON
7514,2013-10-29,76ers,NaN,Jason Richardson,placed on IL recovering from surgery on left k...,True,knee,Leg,13-14
7515,2013-10-29,76ers,NaN,Nerlens Noel,placed on IL recovering from surgery on left k...,True,knee,Leg,13-14
7516,2013-10-29,Bucks,NaN,Carlos Delfino,recovering from surgery on right foot (out ind...,True,foot,Foot,13-14
7517,2013-10-29,Cavaliers,NaN,Carrick Felix,sports hernia (out indefinitely),True,hernia,Others,13-14
7518,2013-10-29,Celtics,NaN,Rajon Rondo,recovering from surgery on left knee to repair...,True,knee,Leg,13-14


In [353]:
## Cargar el archivo de estadisticas de los jugadores
df_gamestats = pd.read_csv('../csv/merged.csv')

In [354]:
df_gamestats = df_gamestats[~df_gamestats['SEASON'].isin(['20-21', '21-22', '22-23', '23-24'])]

In [413]:
# Hay ~500 jugadores por temporada, entonces en total 3727 de 2013-2020
df_gamestats.head()

,PLAYER_ID,PLAYER_NAME,SEASON,SEASON_NUM,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,GP,MIN,USG_PCT,PACE,POSS,FGA_PG,DRIVES,DRIVE_FGA,DRIVE_PASSES,DIST_MILES,AVG_SPEED,PULL_UP_FGA,PULL_UP_FG3A,TOUCHES,FRONT_CT_TOUCHES,AVG_SEC_PER_TOUCH,AVG_DRIB_PER_TOUCH,ELBOW_TOUCHES,POST_TOUCHES,PAINT_TOUCHES,TEAM,INJURED ON,RETURNED,DAYS MISSED,INJURED_TYPE
1851,203932,Aaron Gordon,19-20,19.5,24.0,80,235,62,32.5,0.205,99.72,4182,12.4,4.8,2.0,1.7,2.43,4.2,3.5,1.1,53.0,29.4,2.88,1.91,1.8,3.2,4.3,NaN,NaN,NaN,NaN,NaN
1852,1628988,Aaron Holiday,19-20,19.5,23.0,72,185,66,24.5,0.182,100.31,3398,8.5,7.6,3.2,3.6,1.83,4.2,3.2,1.3,41.4,18.0,4.35,4.17,0.1,0.0,0.3,NaN,NaN,NaN,NaN,NaN
1853,1627846,Abdel Nader,19-20,19.5,26.0,77,225,55,15.8,0.164,101.79,1835,4.8,3.0,1.3,1.1,1.22,4.32,0.6,0.1,15.6,11.1,2.05,1.41,0.2,0.0,0.4,Thunder,2020-01-15,2020-01-29,14.0,Sprained_ankle
1854,1629690,Adam Mokoka,19-20,19.5,21.0,77,190,11,10.2,0.110,100.37,238,2.5,1.1,0.6,0.4,0.79,4.24,0.0,0.0,10.2,7.1,1.47,0.83,0.2,0.0,0.8,NaN,NaN,NaN,NaN,NaN
1855,1629678,Admiral Schofield,19-20,19.5,23.0,77,241,33,11.2,0.118,103.13,791,2.8,0.4,0.1,0.2,0.82,4.19,0.2,0.1,15.7,8.3,1.40,0.51,0.2,0.2,0.6,NaN,NaN,NaN,NaN,NaN


In [356]:
# Unir los dos dataframes por nombre, temporada y equipo
merged_df = pd.merge(df_gamestats, df_injuries_only, how = 'left', left_on=['PLAYER_NAME', 'SEASON', 'TEAM'], right_on=['Relinquished', 'SEASON', 'Team'])

In [414]:
# Eliminar duplicados
merged_df_cleaned = merged_df.drop_duplicates(subset = ['SEASON_NUM', 'injury_type', 'Out Indefinitely', 'PLAYER_NAME', 'Team'])
merged_df_cleaned.head()

,PLAYER_ID,PLAYER_NAME,SEASON,SEASON_NUM,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,GP,MIN,USG_PCT,PACE,POSS,FGA_PG,DRIVES,DRIVE_FGA,DRIVE_PASSES,DIST_MILES,AVG_SPEED,PULL_UP_FGA,PULL_UP_FG3A,TOUCHES,FRONT_CT_TOUCHES,AVG_SEC_PER_TOUCH,AVG_DRIB_PER_TOUCH,ELBOW_TOUCHES,POST_TOUCHES,PAINT_TOUCHES,TEAM,INJURED ON,RETURNED,DAYS MISSED,INJURED_TYPE,Date,Team,Acquired,Relinquished,Notes,Out Indefinitely,injury_type,body_part
0,203932,Aaron Gordon,19-20,19.5,24.0,80,235,62,32.5,0.205,99.72,4182,12.4,4.8,2.0,1.7,2.43,4.2,3.5,1.1,53.0,29.4,2.88,1.91,1.8,3.2,4.3,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1628988,Aaron Holiday,19-20,19.5,23.0,72,185,66,24.5,0.182,100.31,3398,8.5,7.6,3.2,3.6,1.83,4.2,3.2,1.3,41.4,18.0,4.35,4.17,0.1,0.0,0.3,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1627846,Abdel Nader,19-20,19.5,26.0,77,225,55,15.8,0.164,101.79,1835,4.8,3.0,1.3,1.1,1.22,4.32,0.6,0.1,15.6,11.1,2.05,1.41,0.2,0.0,0.4,Thunder,2020-01-15,2020-01-29,14.0,Sprained_ankle,2020-01-14,Thunder,NaN,Abdel Nader,sprained left ankle (DTD),False,ankle,Leg
4,1627846,Abdel Nader,19-20,19.5,26.0,77,225,55,15.8,0.164,101.79,1835,4.8,3.0,1.3,1.1,1.22,4.32,0.6,0.1,15.6,11.1,2.05,1.41,0.2,0.0,0.4,Thunder,2020-01-15,2020-01-29,14.0,Sprained_ankle,2020-08-14,Thunder,NaN,Abdel Nader,placed on IL with neck injury,False,neck,Head
5,1629690,Adam Mokoka,19-20,19.5,21.0,77,190,11,10.2,0.110,100.37,238,2.5,1.1,0.6,0.4,0.79,4.24,0.0,0.0,10.2,7.1,1.47,0.83,0.2,0.0,0.8,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [358]:
clean_list = ['PLAYER_ID', 'Acquired', 'Relinquished']
merged_df_cleaned = merged_df_cleaned.drop(columns = clean_list, index = 1)

In [359]:
merged_df_cleaned.columns

Index(['PLAYER_NAME', 'SEASON', 'SEASON_NUM', 'AGE', 'PLAYER_HEIGHT_INCHES',
       'PLAYER_WEIGHT', 'GP', 'MIN', 'USG_PCT', 'PACE', 'POSS', 'FGA_PG',
       'DRIVES', 'DRIVE_FGA', 'DRIVE_PASSES', 'DIST_MILES', 'AVG_SPEED',
       'PULL_UP_FGA', 'PULL_UP_FG3A', 'TOUCHES', 'FRONT_CT_TOUCHES',
       'AVG_SEC_PER_TOUCH', 'AVG_DRIB_PER_TOUCH', 'ELBOW_TOUCHES',
       'POST_TOUCHES', 'PAINT_TOUCHES', 'TEAM', 'INJURED ON', 'RETURNED',
       'DAYS MISSED', 'INJURED_TYPE', 'Date', 'Team', 'Notes',
       'Out Indefinitely', 'injury_type', 'body_part'],
      dtype='object')

In [360]:
merged_df_cleaned.shape

(4274, 37)

## Dataset final con 4327 filas y 37 columnas

In [361]:
# Binario de si esta fuera indefinidamente
merged_df_cleaned['Out Indefinitely'].fillna(False, inplace=True)
merged_df_cleaned['Out Indefinitely'] = [1 if x == True else 0 for x in merged_df_cleaned['Out Indefinitely']]

C:\Users\PC Daniel\AppData\Local\Temp\ipykernel_16280\3671150297.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df_cleaned['Out Indefinitely'].fillna(False, inplace=True)
C:\Users\PC Daniel\AppData\Local\Temp\ipykernel_16280\3671150297.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df_cl

In [362]:
merged_df_cleaned['Injured'] = np.where(merged_df_cleaned['injury_type'].notna(), 1, 0)

In [363]:
merged_df.head()

,PLAYER_ID,PLAYER_NAME,SEASON,SEASON_NUM,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,GP,MIN,USG_PCT,PACE,POSS,FGA_PG,DRIVES,DRIVE_FGA,DRIVE_PASSES,DIST_MILES,AVG_SPEED,PULL_UP_FGA,PULL_UP_FG3A,TOUCHES,FRONT_CT_TOUCHES,AVG_SEC_PER_TOUCH,AVG_DRIB_PER_TOUCH,ELBOW_TOUCHES,POST_TOUCHES,PAINT_TOUCHES,TEAM,INJURED ON,RETURNED,DAYS MISSED,INJURED_TYPE,Date,Team,Acquired,Relinquished,Notes,Out Indefinitely,injury_type,body_part
0,203932,Aaron Gordon,19-20,19.5,24.0,80,235,62,32.5,0.205,99.72,4182,12.4,4.8,2.0,1.7,2.43,4.2,3.5,1.1,53.0,29.4,2.88,1.91,1.8,3.2,4.3,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1628988,Aaron Holiday,19-20,19.5,23.0,72,185,66,24.5,0.182,100.31,3398,8.5,7.6,3.2,3.6,1.83,4.2,3.2,1.3,41.4,18.0,4.35,4.17,0.1,0.0,0.3,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1627846,Abdel Nader,19-20,19.5,26.0,77,225,55,15.8,0.164,101.79,1835,4.8,3.0,1.3,1.1,1.22,4.32,0.6,0.1,15.6,11.1,2.05,1.41,0.2,0.0,0.4,Thunder,2020-01-15,2020-01-29,14.0,Sprained_ankle,2020-01-14,Thunder,NaN,Abdel Nader,sprained left ankle (DTD),False,ankle,Leg
3,1627846,Abdel Nader,19-20,19.5,26.0,77,225,55,15.8,0.164,101.79,1835,4.8,3.0,1.3,1.1,1.22,4.32,0.6,0.1,15.6,11.1,2.05,1.41,0.2,0.0,0.4,Thunder,2020-01-15,2020-01-29,14.0,Sprained_ankle,2020-01-15,Thunder,NaN,Abdel Nader,placed on IL with sprained left ankle,False,ankle,Leg
4,1627846,Abdel Nader,19-20,19.5,26.0,77,225,55,15.8,0.164,101.79,1835,4.8,3.0,1.3,1.1,1.22,4.32,0.6,0.1,15.6,11.1,2.05,1.41,0.2,0.0,0.4,Thunder,2020-01-15,2020-01-29,14.0,Sprained_ankle,2020-08-14,Thunder,NaN,Abdel Nader,placed on IL with neck injury,False,neck,Head


In [416]:
merged_df_cleaned.dtypes[merged_df_cleaned.dtypes == object]

PLAYER_NAME             object
SEASON                  object
PLAYER_HEIGHT_INCHES    object
PLAYER_WEIGHT           object
DIST_MILES              object
AVG_SPEED               object
PULL_UP_FG3A            object
TEAM                    object
INJURED ON              object
RETURNED                object
INJURED_TYPE            object
Team                    object
Acquired                object
Relinquished            object
Notes                   object
Out Indefinitely        object
injury_type             object
body_part               object
dtype: object

**Procesamiento de datos**

In [365]:
#Mes y año de la columna 'Date'
merged_df_cleaned['year'] = pd.DatetimeIndex(merged_df_cleaned['Date']).year
merged_df_cleaned['month'] = pd.DatetimeIndex(merged_df_cleaned['Date']).month


In [366]:
merged_df_cleaned[['first_name', 'last_name']] = merged_df_cleaned['PLAYER_NAME'].str.split(' ',n=1, expand=True)

In [367]:
df_player = pd.read_csv('../csv/player.csv')

In [368]:
merged_injuries_db = pd.merge(merged_df_cleaned, df_player, on=['first_name', 'last_name'], how='inner')

In [369]:

merged_injuries_db = merged_injuries_db.drop(columns=['TEAM'])

In [370]:
merged_injuries_db.shape

(4284, 44)

In [409]:
merged_injuries_db.head()

,id,PLAYER_NAME,SEASON,SEASON_NUM,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,GP,MIN,USG_PCT,PACE,POSS,FGA_PG,DRIVES,DRIVE_FGA,DRIVE_PASSES,DIST_MILES,AVG_SPEED,PULL_UP_FGA,PULL_UP_FG3A,TOUCHES,FRONT_CT_TOUCHES,AVG_SEC_PER_TOUCH,AVG_DRIB_PER_TOUCH,ELBOW_TOUCHES,POST_TOUCHES,PAINT_TOUCHES,INJURED_ON,RETURNED,DAYS_MISSED,INJURED_TYPE,Date,Team,Notes,Out_Indefinitely,injury_type,body_part,Injured,year,month,first_name,last_name,id_player,full_name,is_active
0,0,Aaron Gordon,19-20,19.5,24.0,80,235,62,32.5,0.205,99.72,4182,12.4,4.8,2.0,1.7,2.43,4.2,3.5,1.1,53.0,29.4,2.88,1.91,1.8,3.2,4.3,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0,NaN,NaN,0,NaN,NaN,Aaron,Gordon,203932,Aaron Gordon,1
1,1,Abdel Nader,19-20,19.5,26.0,77,225,55,15.8,0.164,101.79,1835,4.8,3.0,1.3,1.1,1.22,4.32,0.6,0.1,15.6,11.1,2.05,1.41,0.2,0.0,0.4,2020-01-15,2020-01-29,14.0,Sprained_ankle,2020-01-14,Thunder,sprained left ankle (DTD),0,ankle,Leg,1,2020.0,1.0,Abdel,Nader,1627846,Abdel Nader,0
2,2,Abdel Nader,19-20,19.5,26.0,77,225,55,15.8,0.164,101.79,1835,4.8,3.0,1.3,1.1,1.22,4.32,0.6,0.1,15.6,11.1,2.05,1.41,0.2,0.0,0.4,2020-01-15,2020-01-29,14.0,Sprained_ankle,2020-08-14,Thunder,placed on IL with neck injury,0,neck,Head,1,2020.0,8.0,Abdel,Nader,1627846,Abdel Nader,0
3,3,Adam Mokoka,19-20,19.5,21.0,77,190,11,10.2,0.110,100.37,238,2.5,1.1,0.6,0.4,0.79,4.24,0.0,0.0,10.2,7.1,1.47,0.83,0.2,0.0,0.8,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0,NaN,NaN,0,NaN,NaN,Adam,Mokoka,1629690,Adam Mokoka,0
4,4,Admiral Schofield,19-20,19.5,23.0,77,241,33,11.2,0.118,103.13,791,2.8,0.4,0.1,0.2,0.82,4.19,0.2,0.1,15.7,8.3,1.40,0.51,0.2,0.2,0.6,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0,NaN,NaN,0,NaN,NaN,Admiral,Schofield,1629678,Admiral Schofield,1


In [372]:
#merged_injuries_db.to_csv('../csv/merged_injuries_db.csv', index=False)

In [373]:
# Quiero saber cuantos registros unicos de id hay en la tabla merged_injuries_db
merged_injuries_db['id'].nunique()


1098

In [374]:
# Cambiar columnas a numericas
numeric_columns = ['PLAYER_HEIGHT_INCHES', 'GP', 'PLAYER_WEIGHT', 'POSS', 'DIST_MILES', 'AVG_SPEED', 'PULL_UP_FG3A']
for col in numeric_columns:
  merged_df_cleaned[col] = pd.to_numeric(merged_df_cleaned[col], errors='coerce')

In [375]:
merged_df = pd.merge(df_gamestats, df_injuries_only, how = 'left', left_on=['PLAYER_NAME', 'SEASON', 'TEAM'], right_on=['Relinquished', 'SEASON', 'Team'])
#merged_df_cleaned.to_csv('../csv/merged_injuries1.csv', index=False)

In [376]:
# Esta configuracion nos permite visualizar todas la columnas de nuestro dataframe
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [377]:
merged_injuries_db.dtypes

PLAYER_NAME                     object
SEASON                          object
SEASON_NUM                     float64
AGE                            float64
PLAYER_HEIGHT_INCHES            object
PLAYER_WEIGHT                   object
GP                               int64
MIN                            float64
USG_PCT                        float64
PACE                           float64
POSS                             int64
FGA_PG                         float64
DRIVES                         float64
DRIVE_FGA                      float64
DRIVE_PASSES                   float64
DIST_MILES                      object
AVG_SPEED                       object
PULL_UP_FGA                    float64
PULL_UP_FG3A                    object
TOUCHES                        float64
FRONT_CT_TOUCHES               float64
AVG_SEC_PER_TOUCH              float64
AVG_DRIB_PER_TOUCH             float64
ELBOW_TOUCHES                  float64
POST_TOUCHES                   float64
PAINT_TOUCHES            

In [378]:
merged_injuries_db.head()


,PLAYER_NAME,SEASON,SEASON_NUM,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,GP,MIN,USG_PCT,PACE,POSS,FGA_PG,DRIVES,DRIVE_FGA,DRIVE_PASSES,DIST_MILES,AVG_SPEED,PULL_UP_FGA,PULL_UP_FG3A,TOUCHES,FRONT_CT_TOUCHES,AVG_SEC_PER_TOUCH,AVG_DRIB_PER_TOUCH,ELBOW_TOUCHES,POST_TOUCHES,PAINT_TOUCHES,INJURED ON,RETURNED,DAYS MISSED,INJURED_TYPE,Date,Team,Notes,Out Indefinitely,injury_type,body_part,Injured,year,month,first_name,last_name,id,full_name,is_active
0,Aaron Gordon,19-20,19.5,24.0,80,235,62,32.5,0.205,99.72,4182,12.4,4.8,2.0,1.7,2.43,4.2,3.5,1.1,53.0,29.4,2.88,1.91,1.8,3.2,4.3,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0,NaN,NaN,0,NaN,NaN,Aaron,Gordon,203932,Aaron Gordon,1
1,Abdel Nader,19-20,19.5,26.0,77,225,55,15.8,0.164,101.79,1835,4.8,3.0,1.3,1.1,1.22,4.32,0.6,0.1,15.6,11.1,2.05,1.41,0.2,0.0,0.4,2020-01-15,2020-01-29,14.0,Sprained_ankle,2020-01-14,Thunder,sprained left ankle (DTD),0,ankle,Leg,1,2020.0,1.0,Abdel,Nader,1627846,Abdel Nader,0
2,Abdel Nader,19-20,19.5,26.0,77,225,55,15.8,0.164,101.79,1835,4.8,3.0,1.3,1.1,1.22,4.32,0.6,0.1,15.6,11.1,2.05,1.41,0.2,0.0,0.4,2020-01-15,2020-01-29,14.0,Sprained_ankle,2020-08-14,Thunder,placed on IL with neck injury,0,neck,Head,1,2020.0,8.0,Abdel,Nader,1627846,Abdel Nader,0
3,Adam Mokoka,19-20,19.5,21.0,77,190,11,10.2,0.110,100.37,238,2.5,1.1,0.6,0.4,0.79,4.24,0.0,0.0,10.2,7.1,1.47,0.83,0.2,0.0,0.8,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0,NaN,NaN,0,NaN,NaN,Adam,Mokoka,1629690,Adam Mokoka,0
4,Admiral Schofield,19-20,19.5,23.0,77,241,33,11.2,0.118,103.13,791,2.8,0.4,0.1,0.2,0.82,4.19,0.2,0.1,15.7,8.3,1.40,0.51,0.2,0.2,0.6,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0,NaN,NaN,0,NaN,NaN,Admiral,Schofield,1629678,Admiral Schofield,1


# Dato limpio para el EDA

In [408]:
# pasar el dato limpio a la carpeta de eda para su exploracion 
#merged_df_cleaned.to_csv('../EDA/data/injuries_merged_df_cleaned.csv')

# Creacion de la tabla en la base de datos

In [393]:
%pip install pyodbc
import pyodbc

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [394]:
# Colocamos el nombre de nuestro server y dejamos la database master para que desde allí crear nuestra base de datos
server = r'DESKTOP-S9274BN\SQLEXPRESS'
database = 'nba_henry'

In [395]:
# El nombre de la base de datos a la que le sumaremos nuestras tablas 
database = 'nba_henry'

# Creamos 10 tablas:

# string conexion
conn_str = (


    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=' + server + ';'  
    'DATABASE=' + database + ';' 
    'Trusted_Connection=yes;'

)

# Create a connection
conn = pyodbc.connect(conn_str, autocommit=True)

# Crear un cursor
cursor = conn.cursor()


cursor.execute(
    '''
CREATE TABLE injuries (
    id INT,
    PLAYER_NAME NVARCHAR(255),
    SEASON NVARCHAR(255),
    SEASON_NUM FLOAT,
    AGE FLOAT,
    PLAYER_HEIGHT_INCHES NVARCHAR(255),
    PLAYER_WEIGHT NVARCHAR(255),
    GP INT,
    MIN FLOAT,
    USG_PCT FLOAT,
    PACE FLOAT,
    POSS INT,
    FGA_PG FLOAT,
    DRIVES FLOAT,
    DRIVE_FGA FLOAT,
    DRIVE_PASSES FLOAT,
    DIST_MILES NVARCHAR(255),
    AVG_SPEED NVARCHAR(255),
    PULL_UP_FGA FLOAT,
    PULL_UP_FG3A NVARCHAR(255),
    TOUCHES FLOAT,
    FRONT_CT_TOUCHES FLOAT,
    AVG_SEC_PER_TOUCH FLOAT,
    AVG_DRIB_PER_TOUCH FLOAT,
    ELBOW_TOUCHES FLOAT,
    POST_TOUCHES FLOAT,
    PAINT_TOUCHES FLOAT,
    INJURED_ON NVARCHAR(255),
    RETURNED NVARCHAR(255),
    DAYS_MISSED FLOAT,
    INJURED_TYPE NVARCHAR(255),
    Date DATETIME,
    Team NVARCHAR(255),
    Notes NVARCHAR(255),
    Out_Indefinitely INT,
    injury_type NVARCHAR(255),
    body_part NVARCHAR(255),
    Injured INT,
    year FLOAT,
    month FLOAT,
    first_name NVARCHAR(255),
    last_name NVARCHAR(255),
    id_player INT,
    full_name NVARCHAR(255),
    is_active INT
);
    
'''
    )

cursor.close()
conn.close()


# Cargamos los datos la SqlServer

In [398]:
merged_injuries_db.head()


,PLAYER_NAME,SEASON,SEASON_NUM,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,GP,MIN,USG_PCT,PACE,POSS,FGA_PG,DRIVES,DRIVE_FGA,DRIVE_PASSES,DIST_MILES,AVG_SPEED,PULL_UP_FGA,PULL_UP_FG3A,TOUCHES,FRONT_CT_TOUCHES,AVG_SEC_PER_TOUCH,AVG_DRIB_PER_TOUCH,ELBOW_TOUCHES,POST_TOUCHES,PAINT_TOUCHES,INJURED ON,RETURNED,DAYS MISSED,INJURED_TYPE,Date,Team,Notes,Out Indefinitely,injury_type,body_part,Injured,year,month,first_name,last_name,id,full_name,is_active
0,Aaron Gordon,19-20,19.5,24.0,80,235,62,32.5,0.205,99.72,4182,12.4,4.8,2.0,1.7,2.43,4.2,3.5,1.1,53.0,29.4,2.88,1.91,1.8,3.2,4.3,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0,NaN,NaN,0,NaN,NaN,Aaron,Gordon,203932,Aaron Gordon,1
1,Abdel Nader,19-20,19.5,26.0,77,225,55,15.8,0.164,101.79,1835,4.8,3.0,1.3,1.1,1.22,4.32,0.6,0.1,15.6,11.1,2.05,1.41,0.2,0.0,0.4,2020-01-15,2020-01-29,14.0,Sprained_ankle,2020-01-14,Thunder,sprained left ankle (DTD),0,ankle,Leg,1,2020.0,1.0,Abdel,Nader,1627846,Abdel Nader,0
2,Abdel Nader,19-20,19.5,26.0,77,225,55,15.8,0.164,101.79,1835,4.8,3.0,1.3,1.1,1.22,4.32,0.6,0.1,15.6,11.1,2.05,1.41,0.2,0.0,0.4,2020-01-15,2020-01-29,14.0,Sprained_ankle,2020-08-14,Thunder,placed on IL with neck injury,0,neck,Head,1,2020.0,8.0,Abdel,Nader,1627846,Abdel Nader,0
3,Adam Mokoka,19-20,19.5,21.0,77,190,11,10.2,0.110,100.37,238,2.5,1.1,0.6,0.4,0.79,4.24,0.0,0.0,10.2,7.1,1.47,0.83,0.2,0.0,0.8,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0,NaN,NaN,0,NaN,NaN,Adam,Mokoka,1629690,Adam Mokoka,0
4,Admiral Schofield,19-20,19.5,23.0,77,241,33,11.2,0.118,103.13,791,2.8,0.4,0.1,0.2,0.82,4.19,0.2,0.1,15.7,8.3,1.40,0.51,0.2,0.2,0.6,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0,NaN,NaN,0,NaN,NaN,Admiral,Schofield,1629678,Admiral Schofield,1


In [399]:
merged_injuries_db.rename(columns={'INJURED ON': 'INJURED_ON'}, inplace=True)
merged_injuries_db.rename(columns={'DAYS MISSED': 'DAYS_MISSED'}, inplace=True)
merged_injuries_db.rename(columns={'lower body injury': 'lower_body_injury'}, inplace=True)
merged_injuries_db.rename(columns={'Out Indefinitely': 'Out_Indefinitely'}, inplace=True)
merged_injuries_db.rename(columns={'id': 'id_player'}, inplace=True)


In [400]:
merged_injuries_db.reset_index(inplace=True)
merged_injuries_db.rename(columns={'index': 'id'}, inplace=True)

In [403]:

from sqlalchemy import create_engine
import pyodbc

server = r'DESKTOP-S9274BN\SQLEXPRESS'
database = 'nba_henry'

connection_string = f'mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes'

# Crear el motor de conexión
engine = create_engine(connection_string)

# Mandar los datos a la tabla ya creada en SQL Server
merged_injuries_db.to_sql('injuries', engine, if_exists='append', index=False)

6